# Flights (Arrivals)

In [1]:
import pandas as pd
import requests
from datetime import datetime, date, timedelta

In [6]:
today = datetime.now().date()
tomorrow = today + timedelta(days=1)

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDP/2023-12-08T08:00/2023-12-08T19:59"

querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": "DO_NOT_FORGET_TO_PUT_YOUR_KEY_HERE",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'arrivals': [{'movement': {'airport': {'icao': 'EDDM', 'iata': 'MUC', 'name': 'Munich'}, 'scheduledTime': {'utc': '2023-12-08 11:20Z', 'local': '2023-12-08 12:20+01:00'}, 'quality': ['Basic']}, 'number': 'LH 2166', 'status': 'Expected', 'codeshareStatus': 'Unknown', 'isCargo': False, 'aircraft': {'model': 'Bombardier CRJ900'}, 'airline': {'name': 'Lufthansa', 'iata': 'LH', 'icao': 'DLH'}}, {'movement': {'airport': {'icao': 'EDDF', 'iata': 'FRA', 'name': 'Frankfurt-am-Main'}, 'scheduledTime': {'utc': '2023-12-08 12:30Z', 'local': '2023-12-08 13:30+01:00'}, 'quality': ['Basic']}, 'number': 'LH 158', 'status': 'Expected', 'codeshareStatus': 'Unknown', 'isCargo': False, 'aircraft': {'reg': 'D-AECD', 'modeS': '3C5464', 'model': 'Embraer 190'}, 'airline': {'name': 'Lufthansa', 'iata': 'LH', 'icao': 'DLH'}}, {'movement': {'airport': {'icao': 'HEGN', 'iata': 'HRG', 'name': 'Hurghada'}, 'scheduledTime': {'utc': '2023-12-08 16:45Z', 'local': '2023-12-08 17:45+01:00'}, 'quality': ['Basic']}, 'nu

In [7]:
icao = "EDDP"
resp = response.json()
arriving_flights = response.json()
pd.json_normalize(arriving_flights["arrivals"])

,number,status,codeshareStatus,isCargo,movement.airport.icao,movement.airport.iata,movement.airport.name,movement.scheduledTime.utc,movement.scheduledTime.local,movement.quality,aircraft.model,airline.name,airline.iata,airline.icao,aircraft.reg,aircraft.modeS
0,LH 2166,Expected,Unknown,False,EDDM,MUC,Munich,2023-12-08 11:20Z,2023-12-08 12:20+01:00,[Basic],Bombardier CRJ900,Lufthansa,LH,DLH,NaN,NaN
1,LH 158,Expected,Unknown,False,EDDF,FRA,Frankfurt-am-Main,2023-12-08 12:30Z,2023-12-08 13:30+01:00,[Basic],Embraer 190,Lufthansa,LH,DLH,D-AECD,3C5464
2,DE 239,Unknown,Unknown,False,HEGN,HRG,Hurghada,2023-12-08 16:45Z,2023-12-08 17:45+01:00,[Basic],Airbus A320,Condor,DE,CFG,NaN,NaN
3,LH 160,Expected,Unknown,False,EDDF,FRA,Frankfurt-am-Main,2023-12-08 17:00Z,2023-12-08 18:00+01:00,[Basic],Bombardier CRJ900,Lufthansa,LH,DLH,D-ACNF,3C4DC6
4,TK 1339,Expected,Unknown,False,LTFM,IST,Istanbul,2023-12-08 17:00Z,2023-12-08 18:00+01:00,[Basic],Boeing 737,Turkish,TK,THY,NaN,NaN
5,OS 253,Unknown,Unknown,False,LOWW,VIE,Vienna,2023-12-08 17:40Z,2023-12-08 18:40+01:00,[Basic],Embraer 195,Austrian,OS,AUA,NaN,NaN
6,XR 597,Unknown,Unknown,False,HEGN,HRG,Hurghada,2023-12-08 17:55Z,2023-12-08 18:55+01:00,[Basic],Boeing 737-800,Corendon Europe,XR,CXI,NaN,NaN


In [8]:
arrivals_df = pd.json_normalize(arriving_flights["arrivals"])[["number", "airline.name", "movement.scheduledTime.utc", "movement.airport.name", "movement.airport.icao"]]

In [9]:
arrivals_df = arrivals_df.rename(columns={"number": "flight_number", "airline.name": "airline", "movement.scheduledTime.utc": "arrival_time", "movement.airport.name": "departure_city", "movement.airport.icao": "departure_airport_icao"})

In [10]:
arrivals_df['arrival_airport_icao'] = icao

In [11]:
arrivals_df['data_retrieved_on'] = datetime.now().date()

In [12]:
arrivals_df = arrivals_df[["arrival_airport_icao", "flight_number", "airline", "arrival_time", "departure_city", "departure_airport_icao", "data_retrieved_on"]]
arrivals_df

,arrival_airport_icao,flight_number,airline,arrival_time,departure_city,departure_airport_icao,data_retrieved_on
0,EDDP,LH 2166,Lufthansa,2023-12-08 11:20Z,Munich,EDDM,2023-12-07
1,EDDP,LH 158,Lufthansa,2023-12-08 12:30Z,Frankfurt-am-Main,EDDF,2023-12-07
2,EDDP,DE 239,Condor,2023-12-08 16:45Z,Hurghada,HEGN,2023-12-07
3,EDDP,LH 160,Lufthansa,2023-12-08 17:00Z,Frankfurt-am-Main,EDDF,2023-12-07
4,EDDP,TK 1339,Turkish,2023-12-08 17:00Z,Istanbul,LTFM,2023-12-07
5,EDDP,OS 253,Austrian,2023-12-08 17:40Z,Vienna,LOWW,2023-12-07
6,EDDP,XR 597,Corendon Europe,2023-12-08 17:55Z,Hurghada,HEGN,2023-12-07


# Create a function

In [13]:
def tomorrows_flight_arrivals(icao_list):

  today = datetime.now().date()
  tomorrow = today + timedelta(days=1)

  list_for_arrivals_df = []

  for icao in icao_list:

      times = [["00:00","11:59"],["12:00","23:59"]]

      for time in times:
          url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

          querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

          headers = {
	        "X-RapidAPI-Key": "DO_NOT_FORGET_TO_PUT_YOUR_KEY_HERE",
	        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
          }

          response = requests.get(url, headers=headers, params=querystring)

          flights_resp = response.json()

          arrivals_df = pd.json_normalize(flights_resp["arrivals"])[["number", "airline.name", "movement.scheduledTime.utc", "movement.airport.name", "movement.airport.icao"]]
          arrivals_df = arrivals_df.rename(columns={"number": "flight_number", "airline.name": "airline", "movement.scheduledTime.utc": "arrival_time", "movement.airport.name": "departure_city", "movement.airport.icao": "departure_airport_icao"})
          arrivals_df['arrival_airport_icao'] = icao
          arrivals_df['data_retrieved_on'] = datetime.now().date()
          arrivals_df = arrivals_df[["arrival_airport_icao", "flight_number", "airline", "arrival_time", "departure_city", "departure_airport_icao", "data_retrieved_on"]]

          list_for_arrivals_df.append(arrivals_df)

  return pd.concat(list_for_arrivals_df, ignore_index=True)

In [14]:
icao_list = ["EDDP", "EDDC", "EDDF"]
flights = tomorrows_flight_arrivals(icao_list)
flights

,arrival_airport_icao,flight_number,airline,arrival_time,departure_city,departure_airport_icao,data_retrieved_on
0,EDDP,OS 251,Austrian,2023-12-08 06:25Z,Vienna,LOWW,2023-12-07
1,EDDP,LH 2166,Lufthansa,2023-12-08 11:20Z,Munich,EDDM,2023-12-07
2,EDDP,LH 158,Lufthansa,2023-12-08 12:30Z,Frankfurt-am-Main,EDDF,2023-12-07
3,EDDP,DE 239,Condor,2023-12-08 16:45Z,Hurghada,HEGN,2023-12-07
4,EDDP,LH 160,Lufthansa,2023-12-08 17:00Z,Frankfurt-am-Main,EDDF,2023-12-07
...,...,...,...,...,...,...,...
1731,EDDF,X3 6757,TUIfly,2023-12-08 21:20Z,Marsa Alam,HEMA,2023-12-07
1732,EDDF,LH 121,Lufthansa,2023-12-08 21:05Z,Munich,EDDM,2023-12-07
1733,EDDF,LH 5713,Lufthansa,2023-12-08 21:40Z,Warsaw,EPWA,2023-12-07
1734,EDDF,LH 6965,Lufthansa,2023-12-08 21:40Z,Lisbon,LPPT,2023-12-07


In [106]:
flights['arrival_time'] = pd.to_datetime(flights['arrival_time'])

In [108]:
flights['data_retrieved_on'] = pd.to_datetime(flights['data_retrieved_on'])